# Pipeline of Automated Ontology Construction

## Initialization

In [1]:
import logging
import os
import nest_asyncio
from dotenv import load_dotenv

In [2]:
from ogmyrag.my_logging import configure_logger
from ogmyrag.ontology_construction import (
   StorageConfig,
   OntologyConstructionSystem
)

In [3]:
ontology_construction_logger = configure_logger(name='ontology_construction',log_level=logging.DEBUG, log_file='logs/ontology_construction.log')
openai_logger = configure_logger(name='openai',log_level=logging.INFO, log_file='logs/openai.log')

In [4]:
# Patch event loop to support re-entry in Jupyter

nest_asyncio.apply()

In [5]:
# Load environment variables

load_dotenv(override=True)

mongo_db_uri = os.getenv("MONGO_DB_URI","")

openai_api_key = os.getenv("OPENAI_API_KEY","")

neo4j_uri = os.getenv("NEO4J_URI","")
neo4j_username = os.getenv("NEO4J_USERNAME","")
neo4j_password = os.getenv("NEO4J_PASSWORD","")

if not mongo_db_uri:
    ontology_construction_logger.error("Please set the MONGO_DB_URI environment variable.")

if not openai_api_key:
    ontology_construction_logger.error("Please set the OPENAI_API_KEY environment variable.")

if not neo4j_uri or not neo4j_username or not neo4j_password:
    ontology_construction_logger.error("Please set the NE04J_URI, NE04J_USERNAME, and NE04J_PASSWORD environment variables.")

## Setup Construction Pipeline

### Initialize Ontology Construction System

In [6]:
ontology_config: StorageConfig = {
    'connection_uri': mongo_db_uri,
    'database_name': 'ogmyrag',
    'collection_name': 'ontology'
}

cq_config: StorageConfig = {
    'connection_uri': mongo_db_uri,
    'database_name': 'ogmyrag',
    'collection_name': 'competency_questions'
}

feedback_config: StorageConfig = {
    'connection_uri': mongo_db_uri,
    'database_name': 'ogmyrag',
    'collection_name': 'feedback'
}

In [7]:
try:
   onto_system = OntologyConstructionSystem(
      ontology_purpose="to model the interrelationships and dynamics of companies listed on the Main and ACE Markets in Malaysia, using only information sourced from listing requirements, company prospectuses, and company reports available through Bursa Malaysia. The model emphasizes strategic aspects, including—but not limited to—executives, directors, management team members, major competitors, business partners, suppliers, products and services, as well as location-specific details such as exports, imports, operations, and other relevant activities. This ontology powers a knowledge graph that enables graph-based Retrieval-Augmented Generation (RAG) to assist retail investors in conducting fundamental analysis.",
      ontology_config=ontology_config,
      cq_config=cq_config,
      feedback_config=feedback_config
   )
except Exception as e:
   ontology_construction_logger.error(f"Error while creating ontology construction system: {e}")

### Generate Competency Questions

In [ ]:
await onto_system.generate_competency_questions(
   personality_num=1,
   task_num=10,
   question_num=4,
)

### Construct Ontology

In [8]:
with open('main_market_listing_requirements/chapter_2.txt', 'r') as file:
   source_text = file.read()

source_text_desc = "Chapter 2 of the Main Market Listing Requirements of Bursa Malaysia"

In [ ]:
await onto_system.handle_request(
   source_text=source_text,
   document_desc=source_text_desc,
   document_constraints="NA",
   max_refinement=2
)

2025-05-10 17:37:39,043 - ontology_construction - DEBUG - System Prompt:


You are a non-taxonomic, relationship-driven ontology construction agent. You are provided with a document describing Chapter 2 of the Main Market Listing Requirements of Bursa Malaysia, and your task is to extend the existing ontology using its content. Follow the guidelines, steps, and output format strictly.

Guidelines:
	1. Only extract entities and relationships that directly serve the ontology's purpose: to model the interrelationships and dynamics of companies listed on the Main and ACE Markets in Malaysia, using only information sourced from listing requirements, company prospectuses, and company reports available through Bursa Malaysia. The model emphasizes strategic aspects, including—but not limited to—executives, directors, management team members, major competitors, business partners, suppliers, products and services, as well as location-specific details such as exports, imports, operations, and oth